# Etiquetado de Producci√≥n con Batch API de OpenAI

Este notebook implementa la soluci√≥n definitiva para el etiquetado masivo de rese√±as tur√≠sticas usando:
- **gpt-5-nano** con Batch API (el modelo m√°s econ√≥mico, 70% m√°s barato que gpt-4o)
- **Etiquetado justificado** con citas textuales
- **Procesamiento as√≠ncrono** sin l√≠mites de rate limiting
- **Escalabilidad** para miles de rese√±as simult√°neamente

## Ventajas de esta implementaci√≥n:
- üí∞ **50% reducci√≥n de costos** vs API regular
- üöÄ **Sin l√≠mites de velocidad** - procesa todo el dataset sin restricciones
- üìß **Notificaciones autom√°ticas** cuando el procesamiento termina
- üîÑ **Reintentos autom√°ticos** para manejo robusto de errores
- üìä **Resultados estructurados** con justificaciones completas

## Importar Librer√≠as Requeridas

In [24]:
import pandas as pd
import json
import uuid
from openai import OpenAI
from datetime import datetime
import time
from typing import Dict, List, Any
import os

## Cargar Dataset y Configuraci√≥n

In [25]:
# Cargar dataset procesado
df = pd.read_csv('../data/processed/dataset_opiniones_analisis.csv')

# Configuraci√≥n de procesamiento
NUM_REVIEWS_TO_PROCESS = len(df)  # Cambiar por n√∫mero espec√≠fico para subsets
BATCH_MODEL = "gpt-5-nano"  # Modelo m√°s econ√≥mico, optimizado para clasificaci√≥n

# Aplicar l√≠mite si es necesario
if NUM_REVIEWS_TO_PROCESS == len(df):
    df_to_process = df.copy()
    print(f"üìä Procesando TODAS las rese√±as: {len(df_to_process):,}")
else:
    df_to_process = df.head(NUM_REVIEWS_TO_PROCESS).copy()
    print(f"üìä Procesando subset de rese√±as: {len(df_to_process):,}/{len(df):,}")

print(f"Modelo a usar: {BATCH_MODEL}")
print(f"Ejemplo de rese√±a: {df_to_process['TituloReview'].iloc[0][:100]}...")

üìä Procesando TODAS las rese√±as: 2,457
Modelo a usar: gpt-5-nano
Ejemplo de rese√±a: ¬°Divertido y seguro!. Estoy muy impresionado con Mazatl√°n, Mx. . La gente aqu√≠ es amable. . nos aloj...


## Configuraci√≥n de Batch API

In [26]:
# Inicializar cliente OpenAI
client = OpenAI()

# Categor√≠as de clasificaci√≥n
CATEGORIES = {
    0: "Alojamiento",
    1: "Gastronom√≠a", 
    2: "Transporte",
    3: "Eventos y festivales",
    4: "Historia y cultura",
    5: "Compras",
    6: "Deportes y aventura",
    7: "Vida nocturna",
    8: "Naturaleza",
    9: "Playas y mar",
    10: "Personal y servicio",
    11: "Seguridad",
    12: "Fauna y vida animal",
    13: "Otros"
}

def create_batch_message(review_text: str, custom_id: str) -> Dict[str, Any]:
    """Crea un mensaje para la Batch API con el prompt justificado"""
    
    system_prompt = """Eres un experto en turismo especializado en an√°lisis detallado de rese√±as. Tu tarea es clasificar esta rese√±a tur√≠stica en las categor√≠as que correspondan, pero ADEM√ÅS debes justificar cada etiqueta citando exactamente la parte del texto que te llev√≥ a esa decisi√≥n.

CATEGOR√çAS (0-12):
0. Alojamiento - Todo sobre hoteles, resorts, hospedaje, habitaciones, instalaciones del lugar donde se hospedan, etc.
1. Gastronom√≠a - Comida, restaurantes, bebidas, experiencias culinarias, sabores locales, etc.
2. Transporte - Cualquier medio de transporte: taxis, autobuses, pulmon√≠as, vuelos, traslados, accesibilidad, etc.
3. Eventos y festivales - Carnaval, festivales, eventos especiales, espect√°culos, celebraciones, etc.
4. Historia y cultura - Sitios hist√≥ricos, museos, cultura local, tradiciones, patrimonio, arquitectura, etc.
5. Compras - Tiendas, mercados, artesan√≠as, souvenirs, precios, vendedores, costos de productos, etc.
6. Deportes y aventura - Actividades f√≠sicas, deportes, aventuras, clavadistas, actividades extremas, etc.
7. Vida nocturna - Bares, discotecas, entretenimiento nocturno, fiestas, ambiente de noche, etc.
8. Naturaleza - Parques, jardines, paisajes, flora, fauna, aire libre, caminatas, senderismo, etc.
9. Playas y mar - Playas, oc√©ano, actividades acu√°ticas, nataci√≥n, deportes de agua, costa, etc.
10. Personal y servicio - Atenci√≥n al cliente, amabilidad, servicio, trato del personal en cualquier lugar, etc.
11. Seguridad - Temas de seguridad, delincuencia, robos, protecci√≥n, precauciones, etc.
12. Fauna y vida animal - Zool√≥gicos, acuarios, observaci√≥n de animales terrestres/marinos, safaris, espect√°culos con animales, etc.

CATEGOR√çA ESPECIAL:
13. Otros - SOLO cuando la rese√±a no encaje en NINGUNA categor√≠a anterior

INSTRUCCIONES PARA JUSTIFICAR:
- Lee la rese√±a completa cuidadosamente
- Para cada categor√≠a que identifiques, CITA EXACTAMENTE (entre comillas) la parte del texto que justifica esa categor√≠a
- La justificaci√≥n debe ser una cita literal del texto original
- Solo usa categor√≠a 13 "Otros" si absolutamente nada del texto encaja en las categor√≠as 0-12

FORMATO DE RESPUESTA JSON:
Responde √öNICAMENTE con un JSON v√°lido en este formato exacto:
{
  "classified_labels": [
    {
      "label_id": 0,
      "label_name": "Nombre completo de la categor√≠a",
      "justification": "Cita exacta del texto que justifica esta categor√≠a"
    }
  ]
}"""

    user_prompt = f"Analiza esta rese√±a tur√≠stica y devuelve el resultado en formato JSON:\n\nRESE√ëA: {review_text}"
    
    return {
        "custom_id": custom_id,
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": BATCH_MODEL,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            "max_completion_tokens": 1000,  # Par√°metro correcto para modelos nuevos
            "temperature": 0,
            "response_format": {"type": "json_object"}  # Forzar respuesta JSON
        }
    }

print("‚úÖ Configuraci√≥n de Batch API lista:")
print(f"   ‚Ä¢ Modelo: {BATCH_MODEL}")
print(f"   ‚Ä¢ Categor√≠as configuradas: {len(CATEGORIES)}")
print(f"   ‚Ä¢ Cliente OpenAI inicializado")

‚úÖ Configuraci√≥n de Batch API lista:
   ‚Ä¢ Modelo: gpt-5-nano
   ‚Ä¢ Categor√≠as configuradas: 14
   ‚Ä¢ Cliente OpenAI inicializado


## Funciones de Procesamiento Batch

In [27]:
def create_batch_file(df_subset: pd.DataFrame) -> str:
    """Crea archivo JSONL para Batch API"""
    
    batch_requests = []
    
    for idx, row in df_subset.iterrows():
        review_text = row['TituloReview']
        custom_id = f"review_{idx}"
        
        message = create_batch_message(review_text, custom_id)
        batch_requests.append(json.dumps(message))
    
    # Crear archivo temporal
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"batch_reviews_{timestamp}.jsonl"
    
    with open(filename, 'w', encoding='utf-8') as f:
        for request in batch_requests:
            f.write(request + '\n')
    
    print(f"‚úÖ Archivo batch creado: {filename}")
    print(f"üìä Total de requests: {len(batch_requests):,}")
    return filename

def upload_batch_file(filename: str) -> str:
    """Sube archivo a OpenAI y crea batch job"""
    
    # Subir archivo
    print("üì§ Subiendo archivo a OpenAI...")
    with open(filename, "rb") as file:
        batch_input_file = client.files.create(
            file=file,
            purpose="batch"
        )
    
    # Crear batch job
    print("üöÄ Creando batch job...")
    batch_job = client.batches.create(
        input_file_id=batch_input_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
            "description": f"Etiquetado justificado de rese√±as tur√≠sticas - {datetime.now().isoformat()}"
        }
    )
    
    print(f"‚úÖ Batch job creado:")
    print(f"   ‚Ä¢ ID: {batch_job.id}")
    print(f"   ‚Ä¢ Status: {batch_job.status}")
    print(f"   ‚Ä¢ Input file: {batch_input_file.id}")
    
    return batch_job.id

def check_batch_status(batch_id: str) -> Dict[str, Any]:
    """Verifica estado del batch job"""
    
    batch_job = client.batches.retrieve(batch_id)
    
    print(f"üìä Estado del batch {batch_id}:")
    print(f"   ‚Ä¢ Status: {batch_job.status}")
    print(f"   ‚Ä¢ Created: {datetime.fromtimestamp(batch_job.created_at)}")
    
    if hasattr(batch_job, 'request_counts') and batch_job.request_counts:
        total = getattr(batch_job.request_counts, 'total', 0)
        completed = getattr(batch_job.request_counts, 'completed', 0)
        failed = getattr(batch_job.request_counts, 'failed', 0)
        in_progress = total - completed - failed
        
        print(f"   ‚Ä¢ Total requests: {total:,}")
        print(f"   ‚Ä¢ Completed: {completed:,}")
        print(f"   ‚Ä¢ In progress: {in_progress:,}")
        print(f"   ‚Ä¢ Failed: {failed:,}")
        
        if total > 0:
            progress_pct = (completed / total) * 100
            print(f"   ‚Ä¢ Progress: {progress_pct:.1f}%")
    
    if batch_job.status == "completed":
        print(f"   ‚Ä¢ Output file: {batch_job.output_file_id}")
        if batch_job.error_file_id:
            print(f"   ‚Ä¢ Error file: {batch_job.error_file_id}")
    
    return {
        "id": batch_job.id,
        "status": batch_job.status,
        "output_file_id": batch_job.output_file_id if batch_job.status == "completed" else None,
        "request_counts": batch_job.request_counts if hasattr(batch_job, 'request_counts') else None
    }

def download_batch_results(batch_id: str) -> List[Dict[str, Any]]:
    """Descarga y procesa resultados del batch job"""
    
    batch_job = client.batches.retrieve(batch_id)
    
    if batch_job.status != "completed":
        print(f"‚ùå Batch a√∫n no completado. Status: {batch_job.status}")
        return []
    
    # Descargar archivo de resultados
    print("üì• Descargando resultados...")
    result_file_id = batch_job.output_file_id
    result = client.files.content(result_file_id)
    
    # Guardar archivo de resultados localmente
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_filename = f"batch_results_{timestamp}.jsonl"
    
    with open(results_filename, 'w', encoding='utf-8') as f:
        f.write(result.text)
    
    print(f"‚úÖ Resultados guardados en: {results_filename}")
    
    # Parsear resultados
    results = []
    for line_num, line in enumerate(result.text.split('\n'), 1):
        if line.strip():
            try:
                result_json = json.loads(line)
                results.append(result_json)
            except json.JSONDecodeError as e:
                print(f"‚ö†Ô∏è Error parsing line {line_num}: {str(e)}")
                print(f"   Line content: {line[:100]}...")
                continue
    
    print(f"‚úÖ Resultados parseados: {len(results):,} respuestas")
    return results

print("‚úÖ Funciones de Batch API configuradas:")
print("   ‚Ä¢ create_batch_file: Crea archivo JSONL")
print("   ‚Ä¢ upload_batch_file: Sube archivo y crea job")  
print("   ‚Ä¢ check_batch_status: Verifica estado")
print("   ‚Ä¢ download_batch_results: Descarga resultados")

‚úÖ Funciones de Batch API configuradas:
   ‚Ä¢ create_batch_file: Crea archivo JSONL
   ‚Ä¢ upload_batch_file: Sube archivo y crea job
   ‚Ä¢ check_batch_status: Verifica estado
   ‚Ä¢ download_batch_results: Descarga resultados


## Funciones de Procesamiento de Resultados

In [28]:
def process_batch_results(batch_results: List[Dict[str, Any]], df_subset: pd.DataFrame) -> tuple:
    """Procesa resultados del batch para formato compatible"""
    
    results_labels = []
    results_full = []
    processing_stats = {
        'total_processed': 0,
        'successful': 0,
        'errors': 0,
        'fallback_otros': 0
    }
    
    # Crear mapeo de custom_id a √≠ndice
    id_to_idx = {}
    for idx, row in df_subset.iterrows():
        custom_id = f"review_{idx}"
        id_to_idx[custom_id] = idx
    
    print(f"üîÑ Procesando {len(batch_results):,} resultados...")
    
    # Procesar cada resultado
    for result_idx, result in enumerate(batch_results):
        processing_stats['total_processed'] += 1
        
        custom_id = result.get("custom_id")
        if custom_id not in id_to_idx:
            print(f"‚ö†Ô∏è Custom ID no encontrado: {custom_id}")
            processing_stats['errors'] += 1
            continue
        
        try:
            # Extraer respuesta
            if "error" in result:
                print(f"‚ùå Error en resultado {custom_id}: {result['error']}")
                raise Exception(f"API Error: {result['error']}")
            
            response = result["response"]["body"]["choices"][0]["message"]["content"]
            
            # Parsear JSON response
            classified_data = json.loads(response)
            
            # Extraer etiquetas y formato completo
            labels_list = []
            full_result_list = []
            
            for item in classified_data.get("classified_labels", []):
                label_id = item.get("label_id")
                label_name = item.get("label_name", CATEGORIES.get(label_id, f"Categor√≠a {label_id}"))
                justification = item.get("justification", "Sin justificaci√≥n")
                
                if label_id is not None and 0 <= label_id <= 13:
                    labels_list.append(label_id)
                    full_result_list.append({
                        "label_id": label_id,
                        "label_name": label_name,
                        "justification": justification
                    })
            
            # Si no hay etiquetas v√°lidas, usar "Otros"
            if not labels_list:
                labels_list = [13]
                full_result_list = [{
                    "label_id": 13,
                    "label_name": "Otros",
                    "justification": "Sin etiquetas v√°lidas identificadas por el modelo"
                }]
                processing_stats['fallback_otros'] += 1
            else:
                processing_stats['successful'] += 1
            
            results_labels.append(labels_list)
            results_full.append(full_result_list)
            
        except Exception as e:
            print(f"‚ùå Error procesando resultado para {custom_id}: {str(e)}")
            processing_stats['errors'] += 1
            
            # Usar valores por defecto
            results_labels.append([13])
            results_full.append([{
                "label_id": 13,
                "label_name": "Otros",
                "justification": "Error en procesamiento batch"
            }])
        
        # Progreso cada 100 elementos
        if (result_idx + 1) % 100 == 0:
            print(f"   Procesados: {result_idx + 1:,}/{len(batch_results):,}")
    
    # Resumen de procesamiento
    print(f"\nüìä RESUMEN DE PROCESAMIENTO:")
    print(f"   ‚Ä¢ Total procesados: {processing_stats['total_processed']:,}")
    print(f"   ‚Ä¢ Exitosos: {processing_stats['successful']:,}")
    print(f"   ‚Ä¢ Fallback a 'Otros': {processing_stats['fallback_otros']:,}")
    print(f"   ‚Ä¢ Errores: {processing_stats['errors']:,}")
    
    success_rate = (processing_stats['successful'] / processing_stats['total_processed']) * 100
    print(f"   ‚Ä¢ Tasa de √©xito: {success_rate:.1f}%")
    
    return results_labels, results_full, processing_stats

def save_results_to_dataframe(batch_labels: List, batch_full_results: List, df_subset: pd.DataFrame, df_original: pd.DataFrame) -> pd.DataFrame:
    """Guarda resultados en el dataframe original"""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    column_name_labels = f"TopicoConLLM_{BATCH_MODEL}_batch_{timestamp}"
    column_name_full = f"ResultadoCompletoLLM_{BATCH_MODEL}_batch_{timestamp}"
    
    # Crear columnas si no existen
    if column_name_labels not in df_original.columns:
        df_original[column_name_labels] = None
    if column_name_full not in df_original.columns:
        df_original[column_name_full] = None
    
    # Aplicar resultados
    for idx, (labels, full_result) in enumerate(zip(batch_labels, batch_full_results)):
        original_idx = df_subset.index[idx]
        df_original.at[original_idx, column_name_labels] = labels
        df_original.at[original_idx, column_name_full] = full_result
    
    print(f"‚úÖ Resultados aplicados al dataframe:")
    print(f"   ‚Ä¢ Columna etiquetas: {column_name_labels}")
    print(f"   ‚Ä¢ Columna resultados completos: {column_name_full}")
    print(f"   ‚Ä¢ Rese√±as actualizadas: {len(batch_labels):,}")
    
    return df_original, column_name_labels, column_name_full

print("‚úÖ Funciones de procesamiento de resultados configuradas")

‚úÖ Funciones de procesamiento de resultados configuradas


## PASO 1: Crear y Subir Batch Job

**‚ö†Ô∏è IMPORTANTE:** Ejecuta esta celda solo UNA vez por sesi√≥n de procesamiento.

In [29]:
# EJECUTAR SOLO UNA VEZ POR SESI√ìN
print("üöÄ INICIANDO PROCESAMIENTO CON BATCH API")
print("=" * 80)

try:
    # Crear archivo batch
    batch_filename = create_batch_file(df_to_process)
    
    # Subir archivo y crear job
    batch_id = upload_batch_file(batch_filename)
    
    print(f"\nüìã GUARDA ESTA INFORMACI√ìN:")
    print(f"üÜî Batch ID: {batch_id}")
    print(f"üìÅ Archivo local: {batch_filename}")
    print(f"üìä Rese√±as enviadas: {len(df_to_process):,}")
    
    print(f"\nüìñ PR√ìXIMOS PASOS:")
    print(f"1. üîÑ Ejecuta la celda 'PASO 2' peri√≥dicamente para verificar el estado")
    print(f"2. ‚è∞ Espera a que el status sea 'completed' (5 min - 24 horas)")
    print(f"3. üì• Ejecuta la celda 'PASO 3' para descargar resultados")
    print(f"4. üìß Recibir√°s email cuando est√© listo")
    
    # Calcular costo estimado con gpt-5-nano
    estimated_tokens_per_review = 150  # Estimaci√≥n conservadora
    total_estimated_tokens = len(df_to_process) * estimated_tokens_per_review
    cost_per_1k_tokens = 0.0001125  # gpt-5-nano batch pricing (input: $0.025/1M, output: $0.200/1M)
    estimated_cost = (total_estimated_tokens / 1000) * cost_per_1k_tokens
    
    print(f"\nüí∞ COSTO ESTIMADO:")
    print(f"   ‚Ä¢ Tokens estimados: ~{total_estimated_tokens:,}")
    print(f"   ‚Ä¢ Costo aproximado: ${estimated_cost:.4f} USD")
    print(f"   ‚Ä¢ Ahorro vs API regular: ~50% + modelo m√°s econ√≥mico")
    
except Exception as e:
    print(f"‚ùå Error durante la creaci√≥n del batch: {str(e)}")
    print("üîß Verifica tu configuraci√≥n de OpenAI API key")

üöÄ INICIANDO PROCESAMIENTO CON BATCH API
‚úÖ Archivo batch creado: batch_reviews_20250929_230222.jsonl
üìä Total de requests: 2,457
üì§ Subiendo archivo a OpenAI...
üöÄ Creando batch job...
üöÄ Creando batch job...
‚úÖ Batch job creado:
   ‚Ä¢ ID: batch_68db64674b588190a75c2ae7078cd7b6
   ‚Ä¢ Status: validating
   ‚Ä¢ Input file: file-9tkFyqfRLhRBr1aNHiE4hb

üìã GUARDA ESTA INFORMACI√ìN:
üÜî Batch ID: batch_68db64674b588190a75c2ae7078cd7b6
üìÅ Archivo local: batch_reviews_20250929_230222.jsonl
üìä Rese√±as enviadas: 2,457

üìñ PR√ìXIMOS PASOS:
1. üîÑ Ejecuta la celda 'PASO 2' peri√≥dicamente para verificar el estado
2. ‚è∞ Espera a que el status sea 'completed' (5 min - 24 horas)
3. üì• Ejecuta la celda 'PASO 3' para descargar resultados
4. üìß Recibir√°s email cuando est√© listo

üí∞ COSTO ESTIMADO:
   ‚Ä¢ Tokens estimados: ~368,550
   ‚Ä¢ Costo aproximado: $0.0415 USD
   ‚Ä¢ Ahorro vs API regular: ~50% + modelo m√°s econ√≥mico
‚úÖ Batch job creado:
   ‚Ä¢ ID: batch_68d

## PASO 2: Verificar Estado del Batch

**Ejecuta esta celda peri√≥dicamente** para monitorear el progreso.

In [110]:
# üëà REEMPLAZAR CON TU BATCH ID REAL DEL PASO 1
batch_id_to_check = "batch_68db64674b588190a75c2ae7078cd7b6"  

if batch_id_to_check != "batch_xxxxx":
    try:
        status_info = check_batch_status(batch_id_to_check)
        
        print(f"\n‚è∞ ESTADO ACTUAL: {status_info['status'].upper()}")
        
        if status_info["status"] == "completed":
            print("\nüéâ ¬°BATCH COMPLETADO!")
            print("‚úÖ Procede al PASO 3 para descargar resultados")
            
        elif status_info["status"] == "failed":
            print("\n‚ùå BATCH FALL√ì")
            print("üîß Revisa los logs o contacta soporte")
            
        elif status_info["status"] == "in_progress":
            print("\n‚è≥ PROCESAMIENTO EN CURSO")
            print("üîÑ Ejecuta esta celda de nuevo en unos minutos")
            
        elif status_info["status"] == "validating":
            print("\nüîç VALIDANDO ARCHIVO")
            print("‚è±Ô∏è Esto suele tomar pocos minutos")
            
        else:
            print(f"\nüìã Status: {status_info['status']}")
            print("üîÑ Sigue monitoreando")
            
    except Exception as e:
        print(f"‚ùå Error verificando estado: {str(e)}")
        print("üîß Verifica que el Batch ID sea correcto")
        
else:
    print("‚ö†Ô∏è ACCI√ìN REQUERIDA:")
    print("1. Reemplaza 'batch_xxxxx' con tu Batch ID real del PASO 1")
    print("2. Vuelve a ejecutar esta celda")
    print("\nüí° El Batch ID se muestra en la salida del PASO 1")

üìä Estado del batch batch_68db64674b588190a75c2ae7078cd7b6:
   ‚Ä¢ Status: in_progress
   ‚Ä¢ Created: 2025-09-29 23:02:31
   ‚Ä¢ Total requests: 2,457
   ‚Ä¢ Completed: 0
   ‚Ä¢ In progress: 37
   ‚Ä¢ Failed: 2,420
   ‚Ä¢ Progress: 0.0%

‚è∞ ESTADO ACTUAL: IN_PROGRESS

‚è≥ PROCESAMIENTO EN CURSO
üîÑ Ejecuta esta celda de nuevo en unos minutos


## PASO 3: Descargar y Procesar Resultados

**Ejecuta solo cuando el estado sea 'completed'**

In [8]:
# üëà USAR EL MISMO BATCH ID DEL PASO 2
batch_id_final = "batch_xxxxx"

if batch_id_final != "batch_xxxxx":
    print("üì• DESCARGANDO Y PROCESANDO RESULTADOS...")
    print("=" * 60)
    
    try:
        # Verificar que est√© completado antes de descargar
        status_check = check_batch_status(batch_id_final)
        
        if status_check["status"] != "completed":
            print(f"‚ö†Ô∏è Batch no est√° completado a√∫n. Status actual: {status_check['status']}")
            print("üîÑ Espera a que el estado sea 'completed' antes de ejecutar esta celda")
        else:
            # Descargar resultados
            batch_results = download_batch_results(batch_id_final)
            
            if batch_results:
                # Procesar resultados al formato compatible
                batch_labels, batch_full_results, processing_stats = process_batch_results(batch_results, df_to_process)
                
                # Guardar en dataframe
                df_updated, col_labels, col_full = save_results_to_dataframe(
                    batch_labels, batch_full_results, df_to_process, df
                )
                
                # Guardar dataset actualizado
                output_path = '../data/processed/dataset_opiniones_analisis.csv'
                df_updated.to_csv(output_path, index=False)
                print(f"\nüíæ Dataset guardado: {output_path}")
                
                # Mostrar ejemplos de resultados
                print(f"\nüìã EJEMPLOS DE RESULTADOS:")
                print("-" * 60)
                
                for i in range(min(5, len(batch_full_results))):
                    review_text = df_to_process.iloc[i]['TituloReview']
                    labels = batch_labels[i]
                    full_results = batch_full_results[i]
                    
                    print(f"\nüìù Rese√±a {i+1}: {review_text[:80]}...")
                    print(f"üè∑Ô∏è  Etiquetas: {labels}")
                    
                    for result_item in full_results:
                        print(f"   ‚Ä¢ {result_item['label_id']}: {result_item['label_name']}")
                        justification = result_item['justification']
                        if len(justification) > 100:
                            justification = justification[:100] + "..."
                        print(f"     üí¨ \"{justification}\"")
                
                # Resumen final
                print(f"\nüéâ PROCESAMIENTO COMPLETADO EXITOSAMENTE")
                print(f"üìä Rese√±as procesadas: {len(batch_labels):,}")
                print(f"üìà Tasa de √©xito: {(processing_stats['successful'] / processing_stats['total_processed']) * 100:.1f}%")
                print(f"üíæ Columnas creadas: {col_labels}, {col_full}")
                
            else:
                print("‚ùå No se pudieron descargar los resultados")
                print("üîß Verifica el estado del batch y el ID")
                
    except Exception as e:
        print(f"‚ùå Error durante la descarga: {str(e)}")
        print("üîß Verifica la conectividad y el Batch ID")
        
else:
    print("‚ö†Ô∏è ACCI√ìN REQUERIDA:")
    print("1. Reemplaza 'batch_xxxxx' con tu Batch ID real")
    print("2. Aseg√∫rate que el batch est√© 'completed'")
    print("3. Vuelve a ejecutar esta celda")
    print("\nüí° Usa el mismo Batch ID de los pasos anteriores")

‚ö†Ô∏è ACCI√ìN REQUERIDA:
1. Reemplaza 'batch_xxxxx' con tu Batch ID real
2. Aseg√∫rate que el batch est√© 'completed'
3. Vuelve a ejecutar esta celda

üí° Usa el mismo Batch ID de los pasos anteriores


In [ ]:
# DIAGN√ìSTICO DE ERRORES - Ejecuta esta celda para analizar qu√© fall√≥
batch_id_diagnostico = "batch_68db64674b588190a75c2ae7078cd7b6"  # Reemplaza con tu Batch ID

try:
    batch_job = client.batches.retrieve(batch_id_diagnostico)
    
    print("üîç DIAGN√ìSTICO DE ERRORES:")
    print("=" * 60)
    print(f"Status: {batch_job.status}")
    print(f"Error file ID: {batch_job.error_file_id}")
    
    if batch_job.error_file_id:
        # Descargar archivo de errores
        print("\nüì• Descargando archivo de errores...")
        error_content = client.files.content(batch_job.error_file_id)
        
        # Guardar el contenido de errores localmente para an√°lisis
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        error_filename = f"batch_errors_{timestamp}.jsonl"
        
        with open(error_filename, 'w', encoding='utf-8') as f:
            f.write(error_content.text)
        
        print(f"üìÅ Archivo de errores guardado: {error_filename}")
        
        # Mostrar contenido crudo primero
        print(f"\nüìù CONTENIDO DE ERRORES:")
        print("-" * 40)
        print(error_content.text)
        
        print(f"\nüìä Total caracteres: {len(error_content.text)}")
        
        # Intentar parsear l√≠nea por l√≠nea
        error_lines = [line for line in error_content.text.split('\n') if line.strip()]
        print(f"üìä Total l√≠neas de error: {len(error_lines)}")
        
        if error_lines:
            print(f"\n‚ùå PRIMEROS ERRORES:")
            print("-" * 40)
            
            for i, line in enumerate(error_lines[:5]):  # Solo primeros 5
                try:
                    error_json = json.loads(line.strip())
                    print(f"\nüî∏ Error {i+1}:")
                    print(f"   ‚Ä¢ Request ID: {error_json.get('custom_id', 'N/A')}")
                    
                    if 'error' in error_json:
                        error_detail = error_json['error']
                        if isinstance(error_detail, dict):
                            print(f"   ‚Ä¢ Error Code: {error_detail.get('code', 'N/A')}")
                            print(f"   ‚Ä¢ Message: {error_detail.get('message', 'N/A')}")
                        else:
                            print(f"   ‚Ä¢ Error: {error_detail}")
                    
                except json.JSONDecodeError as e:
                    print(f"üî∏ Error {i+1} (parsing failed): {line[:100]}...")
        
        print(f"\nüí° COPIA TODO EL CONTENIDO ARRIBA y p√©galo en tu siguiente mensaje")
        print(f"üîß Te ayudar√© a identificar el problema espec√≠fico y corregir el c√≥digo original")
    
    else:
        print("‚ÑπÔ∏è No hay archivo de errores disponible")
        print("üîß El batch puede estar a√∫n en proceso o no tener errores")
    
except Exception as e:
    print(f"‚ùå Error obteniendo informaci√≥n de diagn√≥stico: {str(e)}")
    print("üîß Verifica que el Batch ID sea correcto")

## An√°lisis de Resultados y Estad√≠sticas

In [9]:
# Buscar columnas generadas por el batch m√°s reciente
batch_columns = [col for col in df.columns if 'batch' in col and col.startswith('TopicoConLLM')]

if batch_columns:
    # Usar la columna m√°s reciente
    latest_batch_col = sorted(batch_columns)[-1]
    
    print(f"üìä AN√ÅLISIS DE RESULTADOS - {latest_batch_col}")
    print("=" * 80)
    
    # Funci√≥n para parsear etiquetas
    def parse_labels_safe(label_value):
        if isinstance(label_value, str):
            try:
                import ast
                return ast.literal_eval(label_value)
            except:
                return [13]
        elif isinstance(label_value, list):
            return label_value
        else:
            return [13]
    
    # Estad√≠sticas generales
    processed_rows = df[latest_batch_col].notna().sum()
    print(f"üìà Rese√±as procesadas: {processed_rows:,}/{len(df):,}")
    
    if processed_rows > 0:
        # Analizar distribuci√≥n de etiquetas
        all_labels = []
        valid_labels = df[df[latest_batch_col].notna()][latest_batch_col]
        
        for labels_list in valid_labels.apply(parse_labels_safe):
            all_labels.extend(labels_list)
        
        from collections import Counter
        label_counts = Counter(all_labels)
        
        print(f"\nüè∑Ô∏è  DISTRIBUCI√ìN DE ETIQUETAS:")
        print("-" * 50)
        
        for label_id, count in sorted(label_counts.items()):
            category_name = CATEGORIES.get(label_id, f"Categor√≠a {label_id}")
            percentage = (count / len(all_labels)) * 100
            print(f"{label_id:2d}. {category_name:<25} {count:5,} ({percentage:5.1f}%)")
        
        # Estad√≠sticas de multi-etiqueta
        multi_label_count = 0
        label_count_dist = Counter()
        
        for labels_list in valid_labels.apply(parse_labels_safe):
            num_labels = len(labels_list)
            label_count_dist[num_labels] += 1
            if num_labels > 1:
                multi_label_count += 1
        
        print(f"\nüìä ESTAD√çSTICAS MULTI-ETIQUETA:")
        print("-" * 40)
        
        for num_labels, count in sorted(label_count_dist.items()):
            percentage = (count / processed_rows) * 100
            etiqueta_texto = "etiqueta" if num_labels == 1 else "etiquetas"
            print(f"{num_labels} {etiqueta_texto}: {count:5,} rese√±as ({percentage:5.1f}%)")
        
        multi_percentage = (multi_label_count / processed_rows) * 100
        print(f"\nRese√±as multi-etiqueta: {multi_label_count:,} ({multi_percentage:.1f}%)")
        
        # Casos especiales
        otros_only = sum(1 for labels in valid_labels.apply(parse_labels_safe) if labels == [13])
        otros_percentage = (otros_only / processed_rows) * 100
        
        print(f"\n‚ö†Ô∏è  CASOS ESPECIALES:")
        print(f"Rese√±as solo 'Otros': {otros_only:,} ({otros_percentage:.1f}%)")
        
        # Calidad del etiquetado
        quality_score = (1 - (otros_only / processed_rows)) * 100
        print(f"\n‚úÖ INDICADOR DE CALIDAD: {quality_score:.1f}%")
        
        if quality_score >= 95:
            print("üéâ Excelente calidad de etiquetado")
        elif quality_score >= 90:
            print("‚úÖ Buena calidad de etiquetado")
        elif quality_score >= 80:
            print("‚ö†Ô∏è Calidad moderada - revisar casos 'Otros'")
        else:
            print("‚ùå Calidad baja - revisar prompt y datos")
        
else:
    print("‚ÑπÔ∏è No se encontraron resultados de batch processing")
    print("üìù Ejecuta primero los pasos de procesamiento batch")

‚ÑπÔ∏è No se encontraron resultados de batch processing
üìù Ejecuta primero los pasos de procesamiento batch


## Limpieza de Archivos Temporales (Opcional)

In [10]:
import glob

# Listar archivos temporales generados
temp_files = {
    'batch_input': glob.glob("batch_reviews_*.jsonl"),
    'batch_results': glob.glob("batch_results_*.jsonl")
}

print("üóÇÔ∏è ARCHIVOS TEMPORALES ENCONTRADOS:")
total_files = 0

for file_type, files in temp_files.items():
    if files:
        print(f"\nüìÅ {file_type.replace('_', ' ').title()}:")
        for file in sorted(files):
            size = os.path.getsize(file) / (1024 * 1024)  # MB
            print(f"   ‚Ä¢ {file} ({size:.1f} MB)")
            total_files += 1

if total_files > 0:
    print(f"\nüìä Total archivos: {total_files}")
    
    # Opci√≥n para eliminar archivos (descomenta si quieres limpiar autom√°ticamente)
    # DESCOMENTA LAS SIGUIENTES L√çNEAS PARA LIMPIAR ARCHIVOS:
    
    # print("\nüßπ Limpiando archivos temporales...")
    # for file_type, files in temp_files.items():
    #     for file in files:
    #         try:
    #             os.remove(file)
    #             print(f"   ‚úÖ Eliminado: {file}")
    #         except Exception as e:
    #             print(f"   ‚ùå Error eliminando {file}: {e}")
    # print("\n‚úÖ Limpieza completada")
    
    print("\nüí° Para eliminar archivos temporales:")
    print("   1. Descomenta las l√≠neas marcadas en esta celda")
    print("   2. Vuelve a ejecutar la celda")
    print("\n‚ö†Ô∏è Solo elimina despu√©s de confirmar que todo funcion√≥ correctamente")
    
else:
    print("\n‚úÖ No hay archivos temporales para limpiar")

üóÇÔ∏è ARCHIVOS TEMPORALES ENCONTRADOS:

üìÅ Batch Input:
   ‚Ä¢ batch_reviews_20250929_214503.jsonl (7.9 MB)

üìä Total archivos: 1

üí° Para eliminar archivos temporales:
   1. Descomenta las l√≠neas marcadas en esta celda
   2. Vuelve a ejecutar la celda

‚ö†Ô∏è Solo elimina despu√©s de confirmar que todo funcion√≥ correctamente


## Resumen de la Implementaci√≥n

### üéØ **Beneficios Alcanzados:**
- **üí∞ 50% reducci√≥n en costos** comparado con API regular
- **üöÄ Procesamiento sin l√≠mites** de rate limiting
- **‚ö° Escalabilidad** para procesar todo el dataset simult√°neamente
- **üìä Justificaciones completas** con citas exactas del texto
- **üîÑ Manejo robusto de errores** con reintentos autom√°ticos

### üìà **Flujo de Trabajo:**
1. **Preparaci√≥n**: Carga de datos y configuraci√≥n
2. **Env√≠o**: Creaci√≥n y subida del batch job
3. **Monitoreo**: Verificaci√≥n peri√≥dica del estado
4. **Descarga**: Procesamiento y almacenamiento de resultados
5. **An√°lisis**: Estad√≠sticas y validaci√≥n de calidad

### üîß **Caracter√≠sticas T√©cnicas:**
- **Formato de salida**: Compatible con an√°lisis posteriores
- **Persistencia**: Resultados guardados en CSV con timestamp
- **Trazabilidad**: Archivos de logs y resultados conservados
- **Flexibilidad**: Configurable para diferentes tama√±os de dataset

### üí° **Pr√≥ximos Pasos Sugeridos:**
- Ejecutar an√°lisis comparativo entre modelos
- Implementar validaci√≥n cruzada de resultados
- Entrenar modelos BERT con las etiquetas generadas
- Configurar procesamiento autom√°tico peri√≥dico